# Predict Rank of Starcraft players

Given a performance dataset of Starcraft players, we want to develop a model predict player rank.

We will treat this problem as `classification problem`.

In [52]:
import pandas as pd
import numpy as np
from pandas.api.types import CategoricalDtype
from statsmodels.miscmodels.ordinal_model import OrderedModel
import statsmodels.api as sm
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import mean_squared_error
from xgboost import XGBClassifier

from math import sqrt

Load the data

In [53]:
df = pd.read_csv('./data/starcraft_player_data.csv')
df.head()

,GameID,LeagueIndex,Age,HoursPerWeek,TotalHours,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
0,52,5,27,10,3000,143.7180,0.003515,0.000220,7,0.000110,0.000392,0.004849,32.6677,40.8673,4.7508,28,0.001397,6,0.0,0.000000
1,55,5,23,10,5000,129.2322,0.003304,0.000259,4,0.000294,0.000432,0.004307,32.9194,42.3454,4.8434,22,0.001193,5,0.0,0.000208
2,56,4,30,10,200,69.9612,0.001101,0.000336,4,0.000294,0.000461,0.002926,44.6475,75.3548,4.0430,22,0.000745,6,0.0,0.000189
3,57,3,19,20,400,107.6016,0.001034,0.000213,1,0.000053,0.000543,0.003783,29.2203,53.7352,4.9155,19,0.000426,7,0.0,0.000384
4,58,3,32,10,500,122.8908,0.001136,0.000327,2,0.000000,0.001329,0.002368,22.6885,62.0813,9.3740,15,0.001174,4,0.0,0.000019


I want to define some key features because it is faster to take advantage of autocomplete.

In [54]:
# define some constant
GAME_ID = 'GameID'
LEAGUE_INDEX = 'LeagueIndex'
HOURS_PER_WEEK = 'HoursPerWeek'
AGE = 'Age'
TOTAL_HOURS = 'TotalHours'
APM = 'APM'
SELECT_BY_HOTKEYS = 'SelectByHotkeys'
ASSIGN_TO_HOTKEYS = 'AssignToHotkeys'
UNIQUE_HOTKEYS = 'UniqueHotkeys'
MINIMAP_ATTACKS = 'MinimapAttacks'
MINIMAP_RIGHT_CLICKS = 'MinimapRightClicks'
NUM_PACS = 'NumberOfPACs'
GAP_BETWEEN_PACS = 'GapBetweenPACs'
ACTION_LATENCY = 'ActionLatency'
ACTIONS_IN_PACS = 'ActionsInPAC'
TOTAL_MAP_EXPLORED = 'TotalMapExplored'
WORKERS_MADE = 'WorkersMade'
UNIQUE_UNITS_MADE = 'UniqueUnitsMade'
COMPLEX_UNITS_MADE = 'ComplexUnitsMade'
COMPLEX_ABILITY_USED = 'ComplexAbilitiesUsed'

df.drop(GAME_ID, inplace=True, axis=1)

excluded_features = [LEAGUE_INDEX]
visualize_features = [feature for feature in df.columns if feature not in excluded_features]
df[visualize_features].describe()

,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
count,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000
mean,117.046947,0.004299,0.000374,4.364654,0.000098,0.000387,0.003463,40.361562,63.739403,5.272988,22.131664,0.001032,6.534021,0.000059,0.000142
std,51.945291,0.005284,0.000225,2.360333,0.000166,0.000377,0.000992,17.153570,19.238869,1.494835,7.431719,0.000519,1.857697,0.000111,0.000265
min,22.059600,0.000000,0.000000,0.000000,0.000000,0.000000,0.000679,6.666700,24.093600,2.038900,5.000000,0.000077,2.000000,0.000000,0.000000
25%,79.900200,0.001258,0.000204,3.000000,0.000000,0.000140,0.002754,28.957750,50.446600,4.272850,17.000000,0.000683,5.000000,0.000000,0.000000
50%,108.010200,0.002500,0.000353,4.000000,0.000040,0.000281,0.003395,36.723500,60.931800,5.095500,22.000000,0.000905,6.000000,0.000000,0.000020
75%,142.790400,0.005133,0.000499,6.000000,0.000119,0.000514,0.004027,48.290500,73.681300,6.033600,27.000000,0.001259,8.000000,0.000086,0.000181
max,389.831400,0.043088,0.001752,10.000000,0.003019,0.004041,0.007971,237.142900,176.372100,18.558100,58.000000,0.005149,13.000000,0.000902,0.003084


## EDA

1. Let's visualize the distribution of the features in the data

In [55]:
fig = make_subplots(rows = 6, cols = 3)
for idx, feature in enumerate(visualize_features):
    row = idx // 3 + 1
    col = idx % 3 + 1
    fig.append_trace(go.Histogram(x=df[feature], name=feature), row=row, col=col)

fig.update_layout({'height': 700, 'width': 1100, 'title': 'Stacked Historgrams'})
fig.show()

In [56]:
px.histogram(df, x=LEAGUE_INDEX)

This dataset is imbalanced. We don't have a lot of data points of high ranking players.

We see that there is unknown valued in the `Age`, `HoursPerWeek`, `TotalHours`.
We see that the distributions of some features, such as `HoursPerWeek`, `APM`, `AssignToHotkeys`, etc. are skewed right normal distribution.

In [57]:
df[df[AGE] == '?'].head()

,LeagueIndex,Age,HoursPerWeek,TotalHours,APM,SelectByHotkeys,AssignToHotkeys,UniqueHotkeys,MinimapAttacks,MinimapRightClicks,NumberOfPACs,GapBetweenPACs,ActionLatency,ActionsInPAC,TotalMapExplored,WorkersMade,UniqueUnitsMade,ComplexUnitsMade,ComplexAbilitiesUsed
3340,8,?,?,?,189.7404,0.004582,0.000655,4,0.000073,0.000618,0.006291,23.5130,32.5665,4.4451,25,0.002218,6,0.000000,0.0
3341,8,?,?,?,287.8128,0.029040,0.001041,9,0.000231,0.000656,0.005399,31.6416,36.1143,4.5893,34,0.001138,6,0.000058,0.0
3342,8,?,?,?,294.0996,0.029640,0.001076,6,0.000302,0.002374,0.006294,16.6393,36.8192,4.1850,26,0.000987,6,0.000000,0.0
3343,8,?,?,?,274.2552,0.018121,0.001264,8,0.000053,0.000975,0.007111,10.6419,24.3556,4.3870,28,0.001106,6,0.000000,0.0
3344,8,?,?,?,274.3404,0.023131,0.000739,8,0.000622,0.003552,0.005355,19.1568,36.3098,5.2811,28,0.000739,6,0.000000,0.0


I recognize that there are some values in the `Age`, `HoursPerWeek`, and `TotalHours` are missing. Based on my domain knowledge, this variables are important. 
Also, a lot of these players are in high right, so I need to choose a way to fill these mising values. There are some options:
- Using mean / median of the samples, conditioned by a feature (ie: LeagueIndex).
- Logistic regression to fill the value. This requires we assume the target variables and dependent variables have linear relationship.
- Using non-parametric method, such as random forest, to fill missing values.

So, we need to treat the missing values. I choose a simple non-parametric method, random forest, to fillout the missing values.

### Filling missing values

In [58]:
filling_values = [AGE, HOURS_PER_WEEK, TOTAL_HOURS]
dependent_filling_values = [feature for feature in df.columns if feature not in filling_values]

filled_data = df.copy()
for target in filling_values:
    test = filled_data[filled_data[target] == '?'].copy()
    train = filled_data[filled_data[target] != '?'].copy()
    rf = RandomForestRegressor()
    rf.fit(train[dependent_filling_values], train[target])
    predictions = np.round(rf.predict(test[dependent_filling_values]))
    filled_data.loc[test.index, target] = predictions


Since League Index is categorical ordered values, I will convert it to ordinal values.

In [59]:
def preprocess(dataframe, to_numeric_labels, target_var):
    ordinal = CategoricalDtype(categories=[i for i in range(1,9)], ordered=True)
    dataframe[target_var] = dataframe[target_var].astype(ordinal)
    for feature in to_numeric_labels:
        dataframe[feature] = pd.to_numeric(dataframe[feature])
    return dataframe

There is a player with more than 900,000 hours of playing. I believe that it is an outliers, so I have to remove it for better visulization.

In [60]:
target_var = LEAGUE_INDEX
dependent_vars = [var for var in df.columns if var != target_var]
processed_df = preprocess(filled_data, dependent_vars, target_var)
processed_df = processed_df[processed_df[TOTAL_HOURS] < 900000]

In [61]:
fig = make_subplots(rows = 6, cols = 3)
for idx, feature in enumerate(visualize_features):
    row = idx // 3 + 1
    col = idx % 3 + 1
    fig.append_trace(go.Box(x=processed_df[target_var], y=processed_df[feature], name=feature), row=row, col=col)

fig.update_layout({'height': 800, 'width': 1200, 'title': 'Stacked boxplot'})
fig.show()

As we can see in the charts, high rank players tend to have young age, play at a reasonale time per week, but have significantly high playing time, which indicate a routine of playing every week, but not too much of the time. 
Also, higher rank players have higher APM, which translate to other action like using hotkeys to further increase their performance. 

Let see how 8 classes are seperated using PCA.

In [62]:
pca = PCA(n_components=3)
X = processed_df[visualize_features].values
Y = processed_df[LEAGUE_INDEX].values

X = StandardScaler().fit_transform(X)
pca_x = pca.fit_transform(X)
explained_variance = round(sum(pca.explained_variance_ratio_) * 100, 2)
pca_df = pd.DataFrame(data=pca_x, columns=['PC1', 'PC2', 'PC3'])
pca_df[LEAGUE_INDEX] = Y
px.scatter_3d(pca_df, x='PC1', y='PC2', z='PC3', color=LEAGUE_INDEX, title=f'Explain variance: {explained_variance}%', height=800)

With retaining 48.4% of information, we see that the data is not seperatable, except for the highest ranking players.

## Modeling

There are two approach: parametric and non-parametric. Let's try both approach.
- Parametric: We assume that there is a linear relationship between features and target variable. 
- Non-parametric: we use *Random Forest* and *XGBoost*. Our objective is to find the model that have mimimuim Mean Squared Error (MSE). The reason I choose MSE (even though it is more common to use `accuracy` for classification problem) beceause we want to punish the model more for incorrect prediction that is faraway from the true value. For example, if model predicts someone with rank 6 as 5 is still better than predicting as rank 1. This is essentially important because when we do rank matching because letting two people with closer rank playing against each other is better.

#### Seperate train, test, and validation set

Since there is imbalance dataset, we would like to Stratify train test split.

In [63]:
train, test = train_test_split(processed_df, stratify=processed_df[LEAGUE_INDEX], shuffle=True, random_state=42)

### OrderedModel

I notice that the League Index is ordinal. So regular GLM is not good enough. Therefore, I did some research and use Ordered Model to work with the Ordinal values.

We lost so much data, so I think it is better not to remove with k = 3.

In [64]:
drop_features = [LEAGUE_INDEX]
used_features = [feature for feature in processed_df.columns if feature not in drop_features]


orderedModel = OrderedModel(train[target_var], train[used_features])
result = orderedModel.fit(method='bfgs')
result.summary()

Optimization terminated successfully.
         Current function value: 1.340203
         Iterations: 295
         Function evaluations: 303
         Gradient evaluations: 303


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:            LeagueIndex   Log-Likelihood:                -3410.8
Model:                   OrderedModel   AIC:                             6872.
Method:            Maximum Likelihood   BIC:                             7018.
Date:                Sat, 27 May 2023                                         
Time:                        21:17:40                                         
No. Observations:                2545                                         
Df Residuals:                    2520                                         
Df Model:                          25                                         
========================================================================================
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
Age                      0.0087      0.005      1.599      0.110      -0.002       0.019
HoursPerWeek             0.0071      0.002      3.756      0.000       0.003       0.011
TotalHours               0.0002   2.51e-05      6.551      0.000       0.000       0.000
APM                     -0.0010      0.003     -0.385      0.700      -0.006       0.004
SelectByHotkeys         51.4181     15.524      3.312      0.001      20.992      81.844
AssignToHotkeys       1019.3214    126.924      8.031      0.000     770.554    1268.089
UniqueHotkeys            0.0282      0.010      2.710      0.007       0.008       0.049
MinimapAttacks        1271.9561    153.537      8.284      0.000     971.028    1572.884
MinimapRightClicks      35.4356     65.649      0.540      0.589     -93.234     164.105
NumberOfPACs           342.9058     80.662      4.251      0.000     184.811     501.001
GapBetweenPACs          -0.0112      0.002     -5.706      0.000      -0.015      -0.007
ActionLatency           -0.0200      0.003     -7.558      0.000      -0.025      -0.015
ActionsInPAC             0.0459      0.038      1.192      0.233      -0.030       0.121
TotalMapExplored        -0.0084      0.004     -2.173      0.030      -0.016      -0.001
WorkersMade            210.8827     46.830      4.503      0.000     119.098     302.668
UniqueUnitsMade         -0.0315      0.015     -2.135      0.033      -0.060      -0.003
ComplexUnitsMade       384.0370    263.628      1.457      0.145    -132.665     900.739
ComplexAbilitiesUsed   -41.3925    103.403     -0.400      0.689    -244.058     161.273
1/2                     -1.8838      0.415     -4.538      0.000      -2.697      -1.070
2/3                     -0.0510      0.056     -0.909      0.363      -0.161       0.059
3/4                     -0.1673      0.044     -3.818      0.000      -0.253      -0.081
4/5                     -0.0218      0.036     -0.614      0.540      -0.091       0.048
5/6                      0.0746      0.036      2.101      0.036       0.005       0.144
6/7                      0.7296      0.047     15.638      0.000       0.638       0.821
7/8                     -0.6240      0.182     -3.420      0.001      -0.982      -0.266
========================================================================================
"""

The log-likelihood is too high, which mean the model does not fit the data well. Let's try other approach. Additionally, we lost a lot of valuable data if we try to remove outliers using IQR for just 3 features.

In [73]:
predicted = np.argmax(result.model.predict(result.params, exog=test[used_features]), axis=1) + 1
mean_squared_error(predicted, test[LEAGUE_INDEX])

c:\Users\Vinh\anaconda3\envs\python11\Lib\site-packages\statsmodels\miscmodels\ordinal_model.py:419: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



1.0753828032979977

MSE for GLM is 1.075

### Non-parametric approach

For random forest, there is no need to remove outliers because it is robust to outliers. We will treat the ranking as a continuos values. Then round to the nearest integer as prediction of ranking.

#### Random forest

In [66]:
rf = RandomForestRegressor()
dependent_vars = [feature for feature in processed_df.columns if feature != target_var]
rf.fit(train[dependent_vars], train[target_var])
y_pred = np.round(pd.to_numeric(rf.predict(test[dependent_vars])))
y_true = pd.to_numeric(test[target_var])
print('Mean square error:', mean_squared_error(y_true, y_pred))
print('Accuracy:', rf.score(test[dependent_vars], y_true))

Mean square error: 0.9328621908127208
Accuracy: 0.6370412755063435


Root Mean Square Error for Random Forest is: 0.93

List of important features:

In [67]:
sorted(list(zip(rf.feature_names_in_, rf.feature_importances_)), key = lambda x: x[1], reverse=True)

[('APM', 0.30322976078248587),
 ('ActionLatency', 0.23831686991113213),
 ('TotalHours', 0.09557572361975267),
 ('AssignToHotkeys', 0.04799860255889028),
 ('SelectByHotkeys', 0.041694679754135974),
 ('MinimapAttacks', 0.03880547485149006),
 ('GapBetweenPACs', 0.030270776141465634),
 ('WorkersMade', 0.02762700572002933),
 ('NumberOfPACs', 0.026710796121156056),
 ('MinimapRightClicks', 0.02445406899135993),
 ('ActionsInPAC', 0.023692715275807623),
 ('Age', 0.018939565591264185),
 ('TotalMapExplored', 0.018140182419995553),
 ('HoursPerWeek', 0.016640863118777792),
 ('ComplexAbilitiesUsed', 0.015288029477319979),
 ('UniqueHotkeys', 0.01308143691368453),
 ('UniqueUnitsMade', 0.011833859152569464),
 ('ComplexUnitsMade', 0.007699589598682891)]

Try treatment for imbalance data.

In [68]:
from imblearn.over_sampling import SMOTE

smote_train_X, smote_train_y = SMOTE(random_state=42).fit_resample(train[used_features], train[LEAGUE_INDEX])
rf_over = RandomForestRegressor()
rf_over.fit(smote_train_X, smote_train_y)
y_pred = np.round(pd.to_numeric(rf_over.predict(test[dependent_vars])))
y_true = pd.to_numeric(test[target_var])
print('Mean square error:', mean_squared_error(y_true, y_pred))
print('Accuracy:', rf.score(test[dependent_vars], y_true))

Mean square error: 0.9575971731448764
Accuracy: 0.6370412755063435


#### XGBoost

In [69]:
from sklearn.preprocessing import LabelEncoder

processed_df.loc[processed_df.index, 'XGBoostLalbel'] = LabelEncoder().fit_transform(processed_df[target_var])
train, test = train_test_split(processed_df, stratify=processed_df['XGBoostLalbel'], random_state=42)
best_model = XGBClassifier(objective='multi:softmax', learning_rate=0.08105, tree_method='approx', enable_categorical=True, eval_metric='rmse')
best_model.fit(train[dependent_vars], train['XGBoostLalbel'])
y_pred = pd.to_numeric(best_model.predict(test[dependent_vars])) + 1
print('Mean square error:', mean_squared_error(pd.to_numeric(test[target_var]), y_pred))
print('Accuracy:', best_model.score(test[dependent_vars] + 1, y_true))

C:\Users\Vinh\AppData\Local\Temp\ipykernel_28320\4011065060.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Mean square error: 1.0812720848056536
Accuracy: 0.2956419316843345


The XGBoost worse than Random Forest.

## Findings

Younger players tend to have higher rankings. Higher ranking players also tend to have higher APM, select by hotkeys, and assign to hotkeys, as well as unique hotkeys, mini-map right-click, and number of PACs. The gap between PACs and Action Latency is lower for higher ranking players, which may be due to their younger age.

Other features, such as Actions in PACs and Total Map Exploration, do not contribute significantly to the model.

According to the random forest model, Action Latency, APM, and Select by Hotkeys are the top three most important features for determining player rankings.

In conclusion, random forest model works well with the prediction. It has lower training time, lower errors rate, and the results are interpretable. It yields the lowest MSE and highest accuracy.


## Hypothetical

Below are the recommendation I would give to the stakeholders. We have two main parts to consider: ethical and technical.

### Ethical 
1. **Informed Consent**: be tranparent and inform players about data collection process. Players have the right to understand about the purpose, methods, potential risks involved.
2. **Opt-out provision**: Respect the players' decision. They have the right to opt-out their data.
3. **Privacy and Security**: employ industry standard protocol to protect against hackers. Treat players' data with confidentiality and comply with applicable data protection regulation.
4. **Anonymization and aggregation**: prirotize anonymization and aggregations to reduce the risks of reidentification.
5. **Regular ethical reviews**: conduct periodic ethical reviews to data collection, model development and usage are adhere to our principles.

### Data collection
1. **Data quality**: prioritize to collect more accurate data on the important features. Even though there are some techniques to work with imbalanced dataset, it is better to collect data points for higher ranking players (especially Grand Master and Professional League), so we need to collect more data points about them. 
2. **Increase Data Dimensions**: consider collect players' profiles, gameplay styles, races playing ratios, matches' length.
3. **Scability and Sustainability**: considering if the data collections methods are scabable as we collect more data and feed them into the system.
4. **Model retraining**: With more data, there will more cost to store data and more computation resources to retrain the models.